Check Python version

In [ ]:
!python --version

Import libraries

In [ ]:
import json
import yaml
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### **Extract Data**

In [ ]:
# Through online GitHub
file_name = 'https://github.com/ourownstory/neuralprophet-data/raw/main/datasets/air_passengers.csv'

In [ ]:
train_df = pd.read_csv(file_name)

In [ ]:
train_df.shape

In [ ]:
train_df.columns

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

Define the required time and target columns

In [ ]:
time_col = train_df.columns[0]
target_col = train_df.columns[-1]
train_df[time_col] = train_df[time_col].astype(str)

Change target column to float

In [ ]:
train_df[target_col] = train_df[target_col].astype(float)

In [ ]:
train_df.head()

Plot graph

In [ ]:
figsize = (16, 8)
train_df.set_index(time_col).plot(figsize=figsize);

### **Train API**

Train data: convert df to list-of-list

In [ ]:
train_data = []

In [ ]:
for value in train_df.values:
    train_data.append(list(value))

In [ ]:
train_data[:10]

Define Model Request (optional)

In [ ]:
# Default model request
# model_request = {
#     'type': 'meta_lr',  # 'meta_wa'
#     'scorers': ['smape', 'mape'],
#     'params': {
#         'preprocessors': [
#             {'type': 'dartsimputer'},
#             # {'type': 'simpleimputer', 'params': {'strategy': 'mean'}},
#             {'type': 'minmaxscaler'},
#         ],
#         'base_models': [
#             {'type': 'darts_rnn'},
#             {'type': 'darts_lightgbm'},
#             {'type': 'darts_autotheta'},
#             {'type': 'darts_autoarima'},
#             {'type': 'darts_autoets'},
#             # {'type': 'stats_autotheta'},
#             # {'type': 'stats_autoarima'},
#             # {'type': 'stats_autoets'},
#         ],
#     },
# }

# Onboard NeuralProphet customized model request
# np_external_model_request = {
#     'params': {
#       'changepoints_range': 0.2,
#       'epochs': 2,
#       'growth': 'off'
#     },
#     'metrics': [],
#     'type': 'neuralprophet',
# }

# Customized model request
# model_request = {
#     'type': 'meta_lr',  # 'meta_naive', 'meta_wa'
#     'scorers': ['smape', 'mape'],
#     'params': {
#         'preprocessors': [
#             {'type': 'dartsimputer'},
#             # {'type': 'simpleimputer', 'params': {'strategy': 'mean'}},
#             {'type': 'minmaxscaler'},
#         ],
#         'base_models': [
#             {'type': 'darts_naive'},
#             {'type': 'darts_seasonalnaive'},
#             {'type': 'darts_autotheta'},
#             # {'type': 'stats_autotheta'},
#             {'type': 'darts_autoets'},
#             # {'type': 'stats_autoets'},
#             {'type': 'darts_autoarima'},
#             # {'type': 'stats_autoarima'},
#             {'type': 'darts_tbats'},
#             {'type': 'darts_linearregression'},
#             {'type': 'darts_lightgbm',
#              'params': {
#                  'lags': 12,
#                  'output_chunk_length': 6,
#                  'verbose': -1
#             }},
#             {'type': 'darts_rnn',
#              'params': {
#                  'model': 'LSTM',
#                  'hidden_dim': 10,
#                  'n_rnn_layers': 3
#             }},
#             {'type': 'neuralprophet',
#              'external_params': np_external_model_request
#             }  # Onboard NeuralProphet external service
#         ],
#     },
# }

In [ ]:
print(model_request)

This is for JSON model_request

In [ ]:
# file_path = 'model_request.json'
# # For writing the model request to a json file
# with open(file_path, 'r') as file:
#     model_request = json.load(file)

This is for YAML model_request

In [ ]:
file_path = 'model_request.yaml'
# For reading the model request from a yaml file
with open(file_path, 'r') as file:
    model_request = yaml.safe_load(file)

print(model_request)

Train API JSON Payload

In [ ]:
api_json = {
    'data': train_data,
    'model': model_request  # (optional) can be commented out
}

Build URL

In [ ]:
with open('url.yaml', 'r') as file:
    url_dict = yaml.safe_load(file)

In [ ]:
# URL to our SYBIL AWS service
protocol = url_dict['protocol']
host = url_dict['host']
port = url_dict['port']
endpoint = 'train'

url = '%s://%s:%s/%s' % (protocol, host, str(port), endpoint)

In [ ]:
# protocol, host, port

Call endpoint, receive response JSON, write to output file

In [ ]:
%%time
response = requests.post(url, json=api_json)
print(response)
print()

In [ ]:
train_json_out = response.json()  # dict output
train_json_out

### **Forecast API**

Forecast Dates: predict the next 2 years of air passenger data (1961-1962)

In [ ]:
dates = [
    '1961-01-01', '1961-02-01', '1961-03-01', '1961-04-01', '1961-05-01', '1961-06-01',
    '1961-07-01', '1961-08-01', '1961-09-01', '1961-10-01', '1961-11-01', '1961-12-01',
    '1962-01-01', '1962-02-01', '1962-03-01', '1962-04-01', '1962-05-01', '1962-06-01',
    '1962-07-01', '1962-08-01', '1962-09-01', '1962-10-01', '1962-11-01', '1962-12-01',
]

Model from Train API

In [ ]:
model = train_json_out['model']

Forecast API JSON Payload

In [ ]:
api_json = {
    'model': model,
    'data': dates
}

Build URL (same as train except for the endpoint)

In [ ]:
endpoint = 'forecast'

url = '%s://%s:%s/%s' % (protocol, host, str(port), endpoint)

Call endpoint, receive response JSON, write to output file

In [ ]:
%%time
response = requests.post(url, json=api_json)
print(response)
print()

In [ ]:
forecast_json_out = response.json()  # dict output
forecast_json_out

Combine forecast dates and output

In [ ]:
forecast_df = pd.DataFrame(
    data=forecast_json_out['data'],
    columns=[time_col, target_col],
)

In [ ]:
forecast_df.shape

In [ ]:
forecast_df.columns

In [ ]:
forecast_df.head()

In [ ]:
forecast_df.tail()

Plot train and forecast side-by-side

In [ ]:
train_df['color'] = 'b'
train_df.set_index(time_col).plot(figsize=figsize, color=train_df['color']);

In [ ]:
forecast_df['color'] = 'r'
forecast_df.set_index(time_col).plot(figsize=figsize, color=forecast_df['color']);

Combine train and forecast sets

In [ ]:
df = pd.concat([train_df, forecast_df]).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

Plot combined train and forecast sets with the vertical black line as the cut-off

In [ ]:
df.set_index(time_col).plot(figsize=figsize, color=df['color'])
plt.axvline(x=len(train_df), color='black', label='Train/Forecast set cut-off')
plt.text(x=len(train_df)-9, y=forecast_df[target_col].max(), s='Train', fontweight='bold', fontsize=14)
plt.text(x=len(train_df)+1, y=forecast_df[target_col].max(), s='Forecast', fontweight='bold', fontsize=14)
plt.title(label='Monthly Number of Air Passengers (Train: 1949-1960) (Forecast: 1961-1962)', fontweight='bold', fontsize=20)
plt.show();